In [1]:
import os
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [2]:
cd ..

/home/kgh6784/private/project/instance_segmentation


In [3]:
cd mmdetection

/home/kgh6784/private/project/instance_segmentation/mmdetection


In [4]:
import mmcv
import terminaltables
import pycocotools
from pycocotools.coco import COCO
from mmdet.datasets.builder import DATASETS
from mmdet.datasets import build_dataset
from mmdet.datasets.coco import CocoDataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.apis import init_detector, inference_detector
from mmdet.apis import show_result_pyplot
from mmcv.runner import load_checkpoint
from mmdet.apis import set_random_seed
import mmcv

In [5]:
import torch
import pylab
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
dataset_path = '/home/kgh6784/shared/hdd_ext/nvme1/jihun_instance/lg_molecular/dataset'

In [7]:
# 사용할 모데 초기 py 설정
config = './configs/mask_rcnn/mask_rcnn_r50_fpn_1x_coco.py'

# config 불러오기
cfg = mmcv.Config.fromfile(config)

# 데이터셋 지정
cfg.dataset_type = 'CocoDataset'

# data_root는 사용자가 저장한 데이터가 있는 폴더 전까지
data_root = '/home/kgh6784/shared/hdd_ext/nvme1/jihun_instance/lg_molecular/dataset/'

# class 설정
cfg.classes = ('Normal',)

# train, val, test 데이터셋에 대한 type, data_root, ann_file, img_prefix 설정

cfg.data.samples_per_gpu=2
cfg.data.workers_per_gpu=1

cfg.data.train.data_root = data_root 
cfg.data.train.ann_file = 'modify.json' 
#cfg.data.train.ann_file = 'label(polygon)_train.json' 
cfg.data.train.img_prefix = 'train/'
cfg.data.train.classes = cfg.classes

cfg.data.val.data_root = data_root
#cfg.data.val.ann_file = 'label(polygon)_train.json'
cfg.data.train.ann_file = 'modify.json' 
cfg.data.val.img_prefix = 'train/'
cfg.data.val.classes = cfg.classes

cfg.data.test.data_root = data_root
cfg.data.test.ann_file = 'test.json'
cfg.data.test.img_prefix = 'test/'
cfg.data.test.classes = cfg.classes

# 클래스 수 지정
cfg.model.roi_head.bbox_head.num_classes = 1
cfg.model.roi_head.mask_head.num_classes = 1

# 사전 훈련 모델 지정
cfg.load_from = './checkpoint/mask_rcnn_r50_fpn_1x_coco_20200205-d4b0c5d6.pth'
#cfg.resume_from = './work_dirs/lg_cfg2/epoch_10.pth'
# 가중치 저장 위치
cfg.work_dir = './work_dirs/lg_cfg_maskrnew/'

# learning rate
cfg.optimizer.lr = 0.02 / 8

# 로그 출력 시기 설정
cfg.log_config.interval = 1 
cfg.lr_config.policy = 'step'

# 평가 지표로 설정
cfg.evaluation.metric = ['bbox', 'segm']

# 평가 구간 설정

cfg.evaluation.interval = 10

# 체크포인트 구간 설정
cfg.checkpoint_config.interval = 10

# epoch 설정
cfg.runner = dict(type='EpochBasedRunner', max_epochs=20)
cfg.runner.max_epochs = 20
# 결과 재현을 위한 시드값 설정
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
cfg.device='cuda'



In [8]:
cfg.train_pipeline[2]['img_scale'] = (2666, 1600)
cfg.test_pipeline[2]['img_scale'] = (2666, 1600)
cfg.auto_scale_lr = dict(enable=False, base_batch_size=8)

IndexError: list index out of range

In [9]:
print(f'Config:\n{cfg.pretty_text}')

Config:
model = dict(
    type='MaskRCNN',
    backbone=dict(
        type='ResNet',
        depth=50,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch',
        init_cfg=dict(type='Pretrained', checkpoint='torchvision://resnet50')),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[0.0, 0.0, 0.0, 0.0],
            target_stds=[1.0, 1.0, 1.0, 1.0]),
        loss_cls=dict(
            type='CrossEntropyLoss', use_sigmoid=True, loss_wei

In [ ]:
cfg.model

In [ ]:
cfg.evaluation.

In [10]:
datasets = [build_dataset(cfg.data.train)]
model = build_detector(cfg.model)
model.CLASSES = datasets[0].CLASSES

# 훈련
train_detector(model, datasets, cfg, distributed=False, validate=True)

loading annotations into memory...
Done (t=0.91s)
creating index...
index created!


2022-07-29 04:58:24,263 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.


loading annotations into memory...
Done (t=0.81s)
creating index...


2022-07-29 04:58:25,214 - mmdet - INFO - load checkpoint from local path: ./checkpoint/mask_rcnn_r50_fpn_1x_coco_20200205-d4b0c5d6.pth


index created!


...

2022-07-29 09:46:55,056 - mmdet - INFO - Epoch [10][172/260]	lr: 2.500e-04, eta: 5:08:39, time: 1.714, data_time: 1.436, memory: 3444, loss_rpn_cls: 0.0513, loss_rpn_bbox: 0.1101, loss_cls: 0.2832, acc: 88.5742, loss_bbox: 0.3327, loss_mask: 0.2167, loss: 0.9939
2022-07-29 09:46:57,418 - mmdet - INFO - Epoch [10][173/260]	lr: 2.500e-04, eta: 5:08:27, time: 2.361, data_time: 2.027, memory: 3444, loss_rpn_cls: 0.0933, loss_rpn_bbox: 0.1323, loss_cls: 0.4446, acc: 81.9336, loss_bbox: 0.4849, loss_mask: 0.2973, loss: 1.4523
2022-07-29 09:46:59,583 - mmdet - INFO - Epoch [10][174/260]	lr: 2.500e-04, eta: 5:08:15, time: 2.189, data_time: 1.804, memory: 3444, loss_rpn_cls: 0.0781, loss_rpn_bbox: 0.1509, loss_cls: 0.3517, acc: 84.7656, loss_bbox: 0.4451, loss_mask: 0.2649, loss: 1.2908
2022-07-29 09:47:01,378 - mmdet - INFO - Epoch [10][175/260]	lr: 2.500e-04, eta: 5:08:03, time: 1.781, data_time: 1.503, memory: 3444, loss_rpn_cls: 0.1179, loss_rpn_bbox: 0.1353, loss_cls: 0.3421, acc: 86.1328,

2022-07-29 09:47:56,549 - mmdet - INFO - Epoch [10][204/260]	lr: 2.500e-04, eta: 5:02:12, time: 1.718, data_time: 1.437, memory: 3444, loss_rpn_cls: 0.0519, loss_rpn_bbox: 0.1302, loss_cls: 0.3182, acc: 85.7422, loss_bbox: 0.4318, loss_mask: 0.2950, loss: 1.2271
2022-07-29 09:47:58,513 - mmdet - INFO - Epoch [10][205/260]	lr: 2.500e-04, eta: 5:02:01, time: 1.965, data_time: 1.701, memory: 3444, loss_rpn_cls: 0.0661, loss_rpn_bbox: 0.1327, loss_cls: 0.3981, acc: 82.6172, loss_bbox: 0.4349, loss_mask: 0.3069, loss: 1.3387
2022-07-29 09:47:59,639 - mmdet - INFO - Epoch [10][206/260]	lr: 2.500e-04, eta: 5:01:48, time: 1.126, data_time: 0.866, memory: 3444, loss_rpn_cls: 0.0786, loss_rpn_bbox: 0.1004, loss_cls: 0.3375, acc: 84.8633, loss_bbox: 0.3707, loss_mask: 0.2690, loss: 1.1562
2022-07-29 09:48:01,437 - mmdet - INFO - Epoch [10][207/260]	lr: 2.500e-04, eta: 5:01:36, time: 1.798, data_time: 1.535, memory: 3444, loss_rpn_cls: 0.0872, loss_rpn_bbox: 0.1002, loss_cls: 0.2935, acc: 89.2578,

2022-07-29 09:49:04,345 - mmdet - INFO - Epoch [10][236/260]	lr: 2.500e-04, eta: 4:56:00, time: 1.765, data_time: 1.520, memory: 3444, loss_rpn_cls: 0.0726, loss_rpn_bbox: 0.1157, loss_cls: 0.2808, acc: 88.5742, loss_bbox: 0.3461, loss_mask: 0.2366, loss: 1.0518
2022-07-29 09:49:06,335 - mmdet - INFO - Epoch [10][237/260]	lr: 2.500e-04, eta: 4:55:49, time: 1.988, data_time: 1.664, memory: 3444, loss_rpn_cls: 0.0767, loss_rpn_bbox: 0.1241, loss_cls: 0.3139, acc: 85.8398, loss_bbox: 0.3990, loss_mask: 0.2427, loss: 1.1564
2022-07-29 09:49:08,278 - mmdet - INFO - Epoch [10][238/260]	lr: 2.500e-04, eta: 4:55:37, time: 1.944, data_time: 1.642, memory: 3444, loss_rpn_cls: 0.0684, loss_rpn_bbox: 0.1355, loss_cls: 0.3826, acc: 84.5703, loss_bbox: 0.4354, loss_mask: 0.3110, loss: 1.3329
2022-07-29 09:49:10,561 - mmdet - INFO - Epoch [10][239/260]	lr: 2.500e-04, eta: 4:55:26, time: 2.298, data_time: 1.946, memory: 3444, loss_rpn_cls: 0.0854, loss_rpn_bbox: 0.1336, loss_cls: 0.3513, acc: 85.8398,

[                                                  ] 0/5000, elapsed: 0s, ETA:

FileNotFoundError: Caught FileNotFoundError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/torch/utils/data/_utils/worker.py", line 287, in _worker_loop
    data = fetcher.fetch(index)
  File "/opt/conda/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 49, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/opt/conda/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 49, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/kgh6784/private/project/instance_segmentation/mmdetection/mmdet/datasets/custom.py", line 216, in __getitem__
    return self.prepare_test_img(idx)
  File "/home/kgh6784/private/project/instance_segmentation/mmdetection/mmdet/datasets/custom.py", line 259, in prepare_test_img
    return self.pipeline(results)
  File "/home/kgh6784/private/project/instance_segmentation/mmdetection/mmdet/d

In [1]:
img = mmcv.imread('/home/kgh6784/shared/hdd_ext/nvme1/jihun_instance/lg_molecular/dataset/test/0087.jpg')

model.cfg = cfg
result = inference_detector(model, img)
show_result_pyplot(model, img, result)

NameError: name 'mmcv' is not defined

In [ ]:
cd mmdetection

In [ ]:
# 해당하는 폴더에 예측 json 저장 하는 코드입니다.

!python /home/kgh6784/private/project/instance_segmentation/mmdetection/test.py /home/kgh6784/private/project/instance_segmentation/mmdetection/configs/custom/lg_mask_solov2.py /home/kgh6784/private/project/instance_segmentation/mmdetection/work_dirs/lg_cfg_maskrnew/latest.pth --format-only --eval-options "jsonfile_prefix=/home/kgh6784/private/project/instance_segmentation/mmdetection/work_dirs/lg_cfg_maskrnew/"

/home/kgh6784/private/project/instance_segmentation/mmdetection/mmdet/utils/setup_env.py:38: UserWarning: Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed.
  warnings.warn(
/home/kgh6784/private/project/instance_segmentation/mmdetection/mmdet/utils/setup_env.py:48: UserWarning: Setting MKL_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed.
  warnings.warn(
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
load checkpoint from local path: /home/kgh6784/private/project/instance_segmentation/mmdetection/work_dirs/lg_cfg_maskrnew/latest.pth
The model and loaded state dict do not match exactly

size mismatch for backbone.layer1.0.conv1.weight: copying a param

[                                                  ] 0/350, elapsed: 0s, ETA:/opt/conda/lib/python3.8/site-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755903507/work/aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
[>>>>                           ] 51/350, 1.3 task/s, elapsed: 41s, ETA:   239s

In [ ]:
ls

In [ ]:
os.getcwd()

In [ ]:
os.listdir('/home/kgh6784/private/project/instance_segmentation/mmdetection/work_dirs/lg_cfg2/')

In [ ]:
import json

In [ ]:
with open("/home/kgh6784/private/project/instance_segmentation/mmdetection/work_dirs/lg_cfg2/.segm.json") as f:
    out_data2 = json.load(f)

In [ ]:
out_data2[0]

In [ ]:
result

In [ ]:
annFile = '/home/kgh6784/shared/hdd_ext/nvme1/jihun_instance/lg_molecular/dataset/label(polygon)_train.json'

In [ ]:
coco = COCO(annFile)

In [ ]:
cells = coco.loadCats(coco.getCatIds())
cells

In [ ]:
catids = coco.getCatIds(catNms=['Normal'])
print(catids)

In [ ]:
imgids = coco.getImgIds(catIds=catids)
print(imgids)

In [ ]:
img = coco.loadImgs(188)[0]
print(img)

In [ ]:
img['id']

In [ ]:
annids = coco.getAnnIds(imgIds=img['id'], catIds=catids, iscrowd=None)

In [ ]:
anns = coco.loadAnns(annids)

In [ ]:
anns

In [ ]:
ls

In [ ]:
file_path = '/home/kgh6784/shared/hdd_ext/nvme1/jihun_instance/lg_molecular/dataset/train/'+img['file_name']
image_array = cv2.imread(file_path)
image_array = cv2.cvtColor(image_array, cv2.COLOR_BGR2RGB)
plt.imshow(image_array)
#plt.imshow(image_array)

In [ ]:
plt.figure(figsize=(12,14))
plt.imshow(image_array)
plt.axis('off')

coco.showAnns(anns)

In [ ]:
anns[2]['image_id']

In [ ]:
anns[2]['segmentation'] #|[0]

In [ ]:
len(anns)

In [ ]:
green_color = (0, 255, 0)

draw_img = image_array.copy()
polygon_all = []
for i in range(len(anns)):
    polygon_xy = np.array(anns[i]['segmentation'], np.int32)
    polygon_all.append(polygon_xy)
raw_img = cv2.polylines(draw_img, polygon_all, True, (0, 255, 0))

plt.figure(figsize=(12,14))
plt.imshow(draw_img)
plt.axis('off')

In [ ]:
polygon_all.shape

In [ ]:
check_img = image_array.copy()

In [ ]:
rot180.shape

In [ ]:
draw_img.shape

In [ ]:
flip_img = np.flip(check_img, 1)

In [ ]:
flip_img.shape

In [ ]:
plt.imshow(flip_img)

In [ ]:
#rot180 = np.rot90(np.rot90(check_img))
#plt.imshow(rot180)
raw_img = cv2.polylines(flip_img, polygon_all, True, (0, 255, 0))

plt.figure(figsize=(12,14))
plt.imshow(draw_img)
plt.axis('off')

In [ ]:
green_color = (0, 255, 0)

draw_img = image_array.copy()
raw_img = cv2.polylines(draw_img, polygon_all, True, (0, 255, 0))

plt.figure(figsize=(12,14))
plt.imshow(draw_img)
plt.axis('off')

In [ ]:
import json

In [ ]:
json_files = '/home/kgh6784/shared/hdd_ext/nvme1/jihun_instance/lg_molecular/dataset/label(polygon)_modify.json'

In [ ]:
with open(json_files) as json_file:
    json_data = json.load(json_file)

In [ ]:
for i in range(len(json_data['annotations'])):
    json_list = []
    for v in json_data['annotations'][i]['segmentation']:
        if v not in json_list:
            json_list.append(v)
    json_data['annotations'][i]['segmentation'] = json_list

In [ ]:
for i in range(len(json_data['annotations'])):
    json_list = [[]]
    segment_list = json_data['annotations'][i]['segmentation']
    for v in range(len(segment_list)):
        json_list[0].append(segment_list[v][0])
        json_list[0].append(segment_list[v][1])
    json_data['annotations'][i]['segmentation'] = json_list

In [ ]:
ttt = []
for i in range(len(json_data['annotations'])):
    img_id = json_data['annotations'][i]['image_id']
    if img_id not in ttt:
        ttt.append(img_id)

In [ ]:
xx = []
for i in range(len(json_data['annotations'])):
    ids = json_data['annotations'][i]['id']
    xx.append(ids)

In [ ]:
len(xx)

In [ ]:
xx

In [ ]:
json_data['annotations']#[1]['id']

In [ ]:
json_data

In [ ]:
a = [1,2,3,4]
a[:-1]

In [ ]:
json_data['annotations'][0]['segmentation'] =json_data['annotations'][0]['segmentation'][:-1]

In [ ]:
json_data['annotations']

In [ ]:
test = json_data['annotations'][0]['segmentation']

In [ ]:
test[0:3]

In [ ]:
x = [[]]
x[0]

In [ ]:
x = [[]]
for i in range(len(test)):
    x[0].append(test[i][0])
    x[0].append(test[i][1])

In [ ]:
x

In [ ]:
for i in range(len(test)):
    x_polygon = []
    y_polygon = []
    

In [ ]:
with open('/home/kgh6784/shared/hdd_ext/nvme1/jihun_instance/lg_molecular/dataset/modify.json','w') as f:
    json.dump(json_data, f, ensure_ascii=False, indent=4)

In [ ]:
x = 0
for i in range(len(json_data['annotations'])):
    json_data['annotations'][i]['id'] = x
    x += 1

In [ ]:
xx = []
for i in range(len(json_data['annotations'])):
    ids = json_data['annotations'][i]['id']
    xx.append(ids)

In [ ]:
xx